In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv("titanic.csv")

In [3]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [5]:
df.isnull().any().values

array([False, False, False,  True, False, False, False,  True, False,
       False, False,  True,  True, False, False])

In [6]:
from sklearn.impute import SimpleImputer

In [44]:
category=df.select_dtypes(include=["object","bool"])
num= df.select_dtypes(include=["float64","int64"]).drop(columns=["survived"])

In [45]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [46]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder

In [47]:
num_pipe=Pipeline(steps=[
    ("imputen",SimpleImputer()),
    ("scale",MinMaxScaler())
])

In [224]:
# How OneHotEncoder works.
one=OneHotEncoder()
pd.DataFrame(one.fit_transform(df[['embarked']]).toarray()[0:10]).astype(int)

,0,1,2,3
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
5,0,1,0,0
6,0,0,1,0
7,0,0,1,0
8,0,0,1,0
9,1,0,0,0


In [48]:
cat_pipe= Pipeline(steps=[
    ("imputec",SimpleImputer()),
    ("encode",OneHotEncoder(handle_unknown="ignore"))
])

In [49]:
from sklearn.compose import ColumnTransformer

In [52]:
col_tr= ColumnTransformer(transformers=[
    ("num",num_pipe,num.columns),
    ("cat",cat_pipe,category.columns)
])

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
x=df.drop(columns=["survived"])
y=df['survived']

x_train,x_test,y_train,y_test= train_test_split(x,y)

In [141]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

In [142]:
from sklearn.model_selection import GridSearchCV

In [153]:
model= Pipeline(steps=[
    ("prep",col_tr),
    ("ML",VotingClassifier(estimators=[
        ("KNN",KNeighborsClassifier()),
        ("Tree",DecisionTreeClassifier()),
        ("SVC",SVC()),
        ("Logistic",LogisticRegression())
    ]))])

In [154]:
model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputen',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputec',
                                                                   SimpleImputer()),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['sex', 'em

In [228]:
param_space={
    "prep__num__imputen__strategy":["mean",'median'],
    "prep__cat__imputec__strategy":["median","most_frequent"],
    "ML__KNN__n_neighbors":range(1,50,2),
    "ML__Tree__max_depth":range(1,5,2)
}

In [170]:
model_grid= GridSearchCV(model,param_space,cv=5)

In [171]:
model_grid.fit(x_train,y_train)

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

...

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan
  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1.
 nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan
  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1.
 nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan
  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1.
 nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan
  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1.
 nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan
  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1.
 nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan  1.  1. nan nan
  1.  1.]
  category=UserW

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputen',
                                                                                          SimpleImputer()),
                                                                                         ('scale',
                                                                                          MinMaxScaler())]),
                                                                         Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputec',
                                                                                          SimpleImputer()),
     

In [172]:
model_main=model_grid.best_estimator_

In [173]:
model_main.predict(x_test)

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0], dtype=int64)

In [167]:
from sklearn.metrics import accuracy_score

In [168]:
accuracy_score(y_test,model_main.predict(x_test))

1.0

In [226]:
# Creating Model with KNeighborsClassifier
model_KNN= Pipeline(steps=[
    ("prep",col_tr),
    ("ML",KNeighborsClassifier())])

In [236]:
knn_param_space={
    "prep__num__imputen__strategy":["mean",'median'],
    "prep__cat__imputec__strategy":["median","most_frequent"],
}

In [238]:
grid_KNN=GridSearchCV(model_KNN,knn_param_space,cv=5)
grid_KNN.fit(x_train,y_train)

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "c:\users\acer\appdata\local\programs\python\

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputen',
                                                                                          SimpleImputer()),
                                                                                         ('scale',
                                                                                          MinMaxScaler())]),
                                                                         Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputec',
                                                                                          SimpleImputer()),
     

In [239]:
KNN_model= grid_KNN.best_estimator_

In [241]:
accuracy_score(KNN_model.predict(x_train),y_train)

0.9820359281437125